In [1]:
import numpy as np 
import healpy as hp
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import mylib
import camb

In [2]:
nside = 256
npix = 12*nside**2
lmax = 3*nside-1
nmc = 100

ell_input, TT, TE, EE, BB, PP = np.loadtxt('cl.txt', unpack=True)
input_cls = np.array([TT, EE, BB, TE]) 
input_cls /= ell_input*(ell_input+1)/2/np.pi 
for c in input_cls: c[0] = 0 
input_cls = input_cls[:,:lmax+1]

input_cls_noBB = np.copy(input_cls)
input_cls_noBB[2] = np.zeros(lmax+1)

input_cls_noEE = np.copy(input_cls)
input_cls_noEE[1] = np.zeros(lmax+1)
input_cls_noEE[3] = np.zeros(lmax+1)

input_cls_onlyBB = np.copy(input_cls)
input_cls_onlyBB[0] = np.zeros(lmax+1)
input_cls_onlyBB[1] = np.zeros(lmax+1)
input_cls_onlyBB[3] = np.zeros(lmax+1)

# pars = camb.read_ini('https://raw.githubusercontent.com/cmbant/CAMB/master/inifiles/planck_2018.ini')
# results = camb.get_results(pars)
# cl = results.get_total_cls(CMB_unit='muK')
# input_cls = np.array([cl[:,i][:lmax+1] for i in range(4)])
# ls = np.arange(lmax+1)
# input_cls /= ls*(ls+1)/2/np.pi 
# for c in input_cls: c[0] = 0 
# pars.WantTensors=True
# pars.InitPower.set_params(r=0.1)
# results = camb.get_results(pars)
# cl = results.get_total_cls(CMB_unit='muK')
# input_cls2 = np.array([cl[:,i][:lmax+1] for i in range(4)])
# input_cls2 /= ls*(ls+1)/2/np.pi 

# pars.InitPower.set_params(r=0.01)
# results = camb.get_results(pars)
# cl = results.get_total_cls(CMB_unit='muK')
# input_cls3 = np.array([cl[:,i][:lmax+1] for i in range(4)])
# input_cls3 /= ls*(ls+1)/2/np.pi 

/tmp/ipykernel_13000/1338916599.py:8: RuntimeWarning: invalid value encountered in divide
  input_cls /= ell_input*(ell_input+1)/2/np.pi


In [4]:
hp.write_map(f'input_maps/lcdm_nside8.fits', hp.synfast(input_cls, 8), overwrite=True)

setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]


In [5]:
hp.write_map(f'input_maps/empty_nside64.fits', hp.synfast(np.zeros_like(input_cls), 64), overwrite=True)

setting the output map dtype to [dtype('float64'), dtype('float64'), dtype('float64')]


In [4]:
# # plt.plot(ell_input, TT, label='TT')
# # plt.plot(ell_input, EE, label='EE')
# plt.plot(input_cls[2], label='BB (r=0)')
# plt.plot(input_cls3[2], label='BB (r=0.01)')
# plt.plot(input_cls2[2], label='BB (r=0.1)')

# plt.xlim([2,lmax+1])
# plt.loglog()
# plt.legend()
# plt.xlabel('ell')
# plt.ylabel('C_l [uK^2]')
# plt.grid()
# # plt.title('Planck best-fit EE/BB 2018')

In [4]:
# mask_lat = np.zeros(hp.nside2npix(nside))
# th, ph = hp.pix2ang(nside, np.arange(hp.nside2npix(nside)))
# ph[np.where(ph > np.pi)[0]] -= 2 * np.pi
# mask_lat[np.where((th < 2.7) & (th > 1.7)& (ph > -np.pi) & (ph < np.pi) )[0]] = 1

# hp.mollview(mask_lat)
# print(sum(mask_lat)/npix)

In [5]:


# lat_T_noise_props = [10, 1000, -3.5]
# lat_P_noise_props = [10*np.sqrt(2), 700, -1.4]
# Nl_T = mylib.get_Nl(lat_T_noise_props, lmax)
# Nl_P = mylib.get_Nl(lat_P_noise_props, lmax)
# lat_noise_cls = np.array([Nl_T, Nl_P, Nl_P, np.zeros(lmax+1)])

In [6]:



planck_T_noise_props = [25, 50, -1.8]
planck_P_noise_props = [50, 20, -1.4]
Nl_T_planck = mylib.get_Nl(planck_T_noise_props, lmax)
Nl_P_planck = mylib.get_Nl(planck_P_noise_props, lmax)
planck_noise_cls = np.array([Nl_T_planck, Nl_P_planck, Nl_P_planck, np.zeros(lmax+1)])


lb_T_noise_props = [1.25, 0, 0]
lb_P_noise_props = [2.5, 0, 0]
Nl_T_lb = mylib.get_Nl(lb_T_noise_props, lmax)
Nl_P_lb = mylib.get_Nl(lb_P_noise_props, lmax)
lb_noise_cls = np.array([Nl_T_lb, Nl_P_lb, Nl_P_lb, np.zeros(lmax+1)])


fwhm = np.deg2rad(17/60) #sat 145ghz

/scratch/yuyang/filterbin/mylib.py:81: RuntimeWarning: divide by zero encountered in power
  Nl = n * (l/knee)**alpha + n


In [7]:
for i in tqdm(range(nmc)):
    # np.random.seed(i)
    # input_map = hp.synfast(input_cls, nside, fwhm=fwhm,new=True)
    # hp.write_map(f'lcdm_maps/map_{i}.fits', input_map, overwrite=True)

    #Noleakage
    np.random.seed(i)
    input_map_onlyBB = hp.synfast(input_cls_onlyBB, nside, fwhm=fwhm, new=True)
    hp.write_map(f'input_maps/lcdm_BBonly_maps/map_{i}.fits', input_map_onlyBB, overwrite=True)

    #Leakage    
    # input_map = hp.synfast(input_cls_noBB, nside, fwhm=fwhm,new=True)
    # hp.write_map(f'input_maps/lcdm_noBB_maps/map_{i}.fits', input_map, overwrite=True)

    #Planck
    # planck_noise = hp.synfast(planck_noise_cls, nside, new=True)
    # wienered_planck_map = mylib.wiener_filter(input_map+planck_noise, input_cls, planck_noise_cls)
    # hp.write_map(f'input_maps/planck_like_E_maps/map_{i}.fits', wienered_planck_map, overwrite=True)

    #LB
    # lb_noise = hp.synfast(lb_noise_cls, nside, new=True)
    # wienered_lb_map = mylib.wiener_filter(input_map+lb_noise, input_cls, lb_noise_cls)
    # hp.write_map(f'input_maps/lb_like_E_maps/map_{i}.fits', wienered_lb_map, overwrite=True)

    #Transfer function
    # np.random.seed(i+1000)
    # input_map_onlyBB = hp.synfast(input_cls_onlyBB, nside, fwhm=fwhm, new=True)
    # hp.write_map(f'input_maps/lcdm_BBonly_maps/map_{i+1000}.fits', input_map_onlyBB, overwrite=True)

    # lat_noise = hp.synfast(lat_noise_cls, nside, new=True)
    # wienered_lat_map = mask_lat*mylib.wiener_filter(input_map+lat_noise, input_cls, lat_noise_cls)
    # hp.write_map(f'input_maps/lat_like_E_maps/map_{i}.fits', wienered_lat_map, overwrite=True)

100%|█████████████████████████████████████████| 100/100 [00:35<00:00,  2.80it/s]
